# Representativity - Habitat coverage 1.4: TEST WITH JUST CORAL

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [3]:
# Union the dissolved habitat MPA layer with the merged and dissolved CORAL layer

in_feature_1 = "WDPA_30km_dis"
in_feature_2 = "coral_mol_Carib30_dis"

out_feature = "\\MPA_30km_habitat_union_Rep1_4_CORAL_TEST"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Union(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class= output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_30km_habitat_union_Rep1_4_CORAL_TEST'>

In [4]:
# Repair geoemtry on union, calculate area, and export to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"

in_feature = "MPA_30km_habitat_union_Rep1_4_CORAL_TEST"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area_export():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features= in_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )
    
    # Export tables to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = in_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
repair_geo_area_export()


In [5]:
# Convert the output features to a Pandas DataFrame
df_MPA_union = pd.DataFrame.spatial.from_featureclass("MPA_30km_habitat_union_Rep1_4_CORAL_TEST")


In [6]:
# Display the DataFrame
df_MPA_union.head()

,OBJECTID,FID_WDPA_30km_dis,FID_coral_mol_Carib30_dis,Habitat,area,SHAPE
0,1,1,-1,,355.435344,"{""rings"": [[[-7912339.7958, 3691634.6744999997..."
1,2,2,-1,,0.156941,"{""rings"": [[[-9046012.3971, 3366611.0735], [-9..."
2,3,3,-1,,34.697292,"{""rings"": [[[-7983853.1789, 3718837.932499999]..."
3,4,4,-1,,73805.240577,"{""rings"": [[[-8535044.4214, 2221126.579399999]..."
4,5,5,-1,,1.486187,"{""rings"": [[[-7871791.334000001, 2377738.90110..."


In [8]:
# Reminder to double check this code and double check the union by doing a sample intersection with coral

# Make pivot tables
pivot_habitat = df_MPA_union.pivot_table(
     index = "Habitat",
     values = "area",
     aggfunc = "sum"
).reset_index()
pivot_habitat.columns = ["Habitat", "Total_habitat_area"]
print(pivot_habitat)

# pivot_KBA_EBSA = df_MPA_union[df_MPA_union["FID_KBA_EBSA_mol_Carib_dis"] != -1].pivot_table(
#     index = "Subregion",
#     values = "Area",
#     aggfunc = "sum"
# ).reset_index()
# pivot_KBA_EBSA.columns = ["Subregion", "KBA_EBSA_area"]
# #print(pivot_KBA_EBSA)

# pivot_MPA = df_MPA_union[df_MPA_union["FID_WDPA_30km_dis"] != -1].pivot_table(
#     index = "Habitat",
#     values = "area",
#     aggfunc = "sum"
# ).reset_index()
# pivot_MPA.columns = ["Habitat", "MPA_area"]
# print(pivot_MPA)

pivot_MPA_habitat_int = df_MPA_union[(df_MPA_union["FID_WDPA_30km_dis"] != -1) & (df_MPA_union["FID_coral_mol_Carib30_dis"] != -1)].pivot_table(
    index = "Habitat",
    values = "area",
    aggfunc = "sum"
).reset_index()
pivot_MPA_habitat_int.columns = ["Habitat", "MPA_habitat_int_area"]
print(pivot_MPA_habitat_int)

# Merge the pivot tables into a single pivot table. The merge should be based on Habitat
habitat_coverage = pd.merge(pivot_habitat, pivot_MPA_habitat_int, on="Habitat", how="left", validate = "1:1")
print(habitat_coverage)

# KBA_EBSA_coverage = pd.merge(KBA_EBSA_coverage, pivot_KBA_EBSA, on="Subregion", how="left", validate = "1:1")
# KBA_EBSA_coverage = pd.merge(KBA_EBSA_coverage, pivot_MPA_KBA_EBSA_int, on="Subregion", how="left", validate = "1:1")


#add column for the percent MPA coverage per habitat
habitat_coverage.insert(3, "%MPA_coverage", habitat_coverage["MPA_habitat_int_area"] / habitat_coverage["Total_habitat_area"] * 100)
print(habitat_coverage)


  Habitat  Total_habitat_area
0               693322.377991
1   coral        13308.274663
  Habitat  MPA_habitat_int_area
0   coral           5640.664727
  Habitat  Total_habitat_area  MPA_habitat_int_area
0               693322.377991                   NaN
1   coral        13308.274663           5640.664727
  Habitat  Total_habitat_area  MPA_habitat_int_area  %MPA_coverage
0               693322.377991                   NaN            NaN
1   coral        13308.274663           5640.664727      42.384643


In [10]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"
table = habitat_coverage
output_name = "MPA_30km_habitat_union_Rep1_4_CORAL_TEST_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))